## 用Mnist数据集进行分类算法

In [1]:
# 防止内核挂掉
import os
os.environ["KMP_DUPLICATE_LIB_OK"]  =  "TRUE"

In [2]:
import numpy as np
from torch import nn, optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

导入数据集

In [3]:
# 训练集
train_dataset = datasets.MNIST(root='./data/', 
                               train=True, 
                               transform=transforms.ToTensor(), 
                               download=True)

# 测试集
test_dataset = datasets.MNIST(root='./data/', 
                               train=False, 
                               transform=transforms.ToTensor(), 
                               download=True)

批次设置

In [4]:
# 批次大小
batch_size = 64 # 每次传入数据的量

# 装载训练集
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True) # 打乱数据集

# 装载测试集
test_loader = DataLoader(dataset=test_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

查看第一个loader中的数据

In [5]:
inputs, labels = next(iter(train_loader))
print('input shape: ', inputs.shape)
print('label shape: ', labels.shape)
print('labels: ', labels)

input shape:  torch.Size([64, 1, 28, 28])
label shape:  torch.Size([64])
labels:  tensor([8, 4, 4, 1, 9, 5, 7, 7, 4, 1, 4, 9, 7, 9, 1, 4, 4, 1, 4, 9, 5, 0, 6, 0,
        7, 1, 6, 1, 5, 4, 4, 7, 4, 3, 6, 4, 2, 0, 3, 1, 2, 1, 1, 8, 8, 1, 5, 7,
        1, 3, 8, 8, 7, 0, 7, 4, 7, 8, 4, 9, 7, 6, 0, 4])


### 网络结构

In [6]:
# 定义网络结构
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 10)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        # 首先需要给数据进行处理，将原来[64, 1, 28, 28](样本数，通道数，长，宽)变为[64, 728]的形状
        # 全连接层输入必须是二维数据
        x = x.view(x.size()[0], -1)
        
        layer1 = self.fc1(x)
        layer2 = self.softmax(layer1)
        
        return layer2

### 模型参数

In [7]:
# 定义模型
model = Net()

# 损失函数
crossentropy_loss = nn.CrossEntropyLoss()

# 优化器
optimizer = optim.SGD(model.parameters(), lr=0.5)

### 训练与测试的方法

In [8]:
# 训练的方法
def train():
    for i, data in enumerate(train_loader):
        # 获取一个批次的数据与标签
        inputs, labels = data
        # 数据传给模型，得到预测结果
        out = model(inputs)
        # 计算loss
        loss = crossentropy_loss(out, labels) # 交叉熵不用one-hot编码
        # 梯度清零
        optimizer.zero_grad()
        # 计算梯度
        loss.backward()
        # 修改权值
        optimizer.step()

In [9]:
# 测试的方法
def test():
    correct = 0
    for i, data in enumerate(test_loader):
        # 获取一个批次的数据与标签
        inputs, labels = data
        # 数据传给模型，得到预测结果
        out = model(inputs)
        # 获得最大值与其所在位置
        _, predicted = torch.max(out, 1)
        # 预测正确的个数
        correct += (predicted == labels).sum()
    
    print(f'Test acc: {correct.item()/len(test_dataset)}')

In [10]:
for epoch in range(10):
    print(f'epoch: {epoch}')
    train()
    test()

epoch: 0
Test acc: 0.906
epoch: 1
Test acc: 0.9117
epoch: 2
Test acc: 0.9173
epoch: 3
Test acc: 0.9192
epoch: 4
Test acc: 0.9213
epoch: 5
Test acc: 0.9222
epoch: 6
Test acc: 0.9218
epoch: 7
Test acc: 0.9233
epoch: 8
Test acc: 0.9238
epoch: 9
Test acc: 0.9233
